## Build custom container for distributed training with Horovod and MXNet

Sagemaker provides a pre-built Deep Learning containers for serving and training tasks. These containers come with installed and configured software such as python packages, NVIDIA drivers and toolkits etc. We'll use [training image with MXNet 1.6, py3, and CUDA 10.1](https://github.com/aws/sagemaker-mxnet-container/blob/master/docker/1.6.0/py3/Dockerfile.gpu) as the base image. Additionally, we will copy training code, define required Sagemaker, and configure ssh communication.


In [2]:
! pygmentize Dockerfile

# Base image: https://github.com/aws/sagemaker-mxnet-container/blob/master/docker/1.6.0/py3/Dockerfile.gpu
FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/mxnet-training:1.6.0-gpu-py36-cu101-ubuntu16.04
LABEL author="vadimd@amazon.com"

RUN pip install gluoncv

########### Sagemaker setup ##########
COPY container_training /opt/ml/code
WORKDIR /opt/ml/code

ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
ENV SAGEMAKER_PROGRAM hvd_launcher.py

########### OpenSHH Config for MPI ##########
RUN mkdir -p /var/run/sshd && \
  sed 's@session\s*required\s*pam_loginuid.so@session optional pam_loginuid.so@g' -i /etc/pam.d/sshd

RUN rm -rf /root/.ssh/ && \
  mkdir -p /root/.ssh/ && \
  ssh-keygen -q -t rsa -N '' -f /root/.ssh/id_rsa && \
  cp /root/.ssh/id_rsa.pub /root/.ssh/authorized_keys


Execute cells bellow to loging to remote ECR (which hosts base image) and private ECR where training container will be pushed to.

In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-2.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 553020858742.dkr.ecr.us-east-2.amazonaws.com

Now let's build and push custom container for MXNet distributed training.

In [ ]:
! ./build_and_push.sh mxnet-distributed latest

## Define common parameters

Execute cells below to do necessary imports and basic configuration of Sagemaker training job

In [5]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.mxnet import MXNet
import sagemaker

role = get_execution_role()

In [6]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'mxnet-distr-input'
prefix_output = 'mxnet-distr-ouput'

In [7]:
container = "mxnet-distributed" # your container name
tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container, tag)

print("Following Sagemaker container will be used for training: ", image)

Following Sagemaker container will be used for training:  553020858742.dkr.ecr.us-east-2.amazonaws.com/mxnet-distributed:latest


## Review training sources

There are two scripts in container_training folders which will be copied to training container:
- `hvd_launcher.py` captures configuration of Sagemaker training cluster and spawns training processes on MPI cluster. Sagemaker starts training by running command like this on all training nodes: `python hvd_lancher.py -train_script value1 -train_param1 value1 ...`
- `distributed_mnist.py` is actual training script which uses Horovod classes to coordinate training processes across multiple nodes. You can add another training script in `container_training` folder and provide its name in `train-script` Sagemaker hyperparameter.

In [ ]:
! pygmentize container_training/hvd_launcher.py

In [ ]:
! pygmentize container_training/distributed_mnist.py

## Start training job

Define hyperparameters of training hob. Note, that `train-script` param define training script which will be executed on Horovod distirbuted cluster. Additionally, you can also define any parameters of your training script.

In [3]:
hyperparameters = {
    "train-script" : "distributed_mnist.py",
    
    # Below you can add args which will passed directly to training script
    "epochs" : 60, 
    "batch-size" : 32
}

In [8]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    train_instance_count=2,
                                    train_instance_type='ml.p3.16xlarge',
#                                     train_instance_type='local_gpu',
                                    sagemaker_session = sess,
                                    hyperparameters = hyperparameters
                                   )

est.fit(wait=False)